# rag-search


## Import packages

In [1]:
import docx
import PyPDF2
from pptx import Presentation
from os import listdir
from os.path import isfile, join, isdir
from langchain_text_splitters import TokenTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_qdrant import Qdrant

In [2]:
def list_files(dir):
    # Initialize a empty list to store the file path
    arquivo_list = []
    
    # Iterate all files and directories on specif directory
    for f in listdir(dir):
        
        # If it's a file, add to the list
        if isfile(join(dir, f)):
            arquivo_list.append(join(dir, f))
        
        # If it is a directory, call the function recursively and add the results to the list
        elif isdir(join(dir, f)):
            arquivo_list += list_files(join(dir, f))
    
    # Return the list of files
    return arquivo_list

In [5]:

# Define the function that loads the text from a Word file 
def load_docx_text(arquivoname):
    
    # Open the Word file
    doc = docx.Document(arquivoname)
    
    # Extract the text from each paragraph and add it to the list
    fullText = [para.text for para in doc.paragraphs]
    
    
    # Join all texts into a single string separated by line breaks
    return '\n'.join(fullText)


In [4]:
# Define the function that load the text from a PowerPoint file
def load_pptx_text(arquivoname):
    
    # Open the PowerPoint file
    prs = Presentation(arquivoname)
    
    # Initialize a empty list to store the texts
    fullText = []
    
    # Iterate all slides
    for slide in prs.slides:
        
        # Iterate all shapes on slides
        for shape in slide.shapes:
            
            # If the shape has the "text" attributes, add it to the list 
            if hasattr(shape, "text"):
                fullText.append(shape.text)
    
    # Join all texts into a single string separated by line breaks
    return '\n'.join(fullText)